# Mod 5 Project - Bank Marketing Classifier

Student details:
* Student name: **Ryan Beck** 
* Student pace: **part time** 
* Scheduled project review date/time: 
* Instructor name: **Abhineet Kulkarni**
* Blog post URL:

## Project Overview: 

# Import Libraries and Data

## Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
plt.style.use('fivethirtyeight')

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

from sklearn.model_selection import GridSearchCV

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

import xgboost as xgb

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, roc_curve, auc

from itertools import combinations

from sklearn.feature_selection import RFE

## Dataset - Bank Marketing Data Set

### Abstract: 
The data is related with direct marketing campaigns (phone calls) of a Portuguese banking institution. **The classification goal** is to predict if the client will subscribe a term deposit (variable y).

### Citation:
  This dataset is publicly available for research. The details are described in [Moro et al., 2014]. 
  Please include this citation if you plan to use this database:

  [Moro et al., 2014] S. Moro, P. Cortez and P. Rita. A Data-Driven Approach to Predict the Success of Bank Telemarketing. Decision Support Systems, In press, http://dx.doi.org/10.1016/j.dss.2014.03.001

  Available at: [pdf] http://dx.doi.org/10.1016/j.dss.2014.03.001
                [bib] http://www3.dsi.uminho.pt/pcortez/bib/2014-dss.txt

### Input variables:

_**Bank client data:**_
1. **age**: (numeric)
2. **job**: type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3. **marital**: marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4. **education**: (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5. **default**: has credit in default? (categorical: 'no','yes','unknown')
6. **housing**: has housing loan? (categorical: 'no','yes','unknown')
7. **loan**: has personal loan? (categorical: 'no','yes','unknown')

_**Related with the last contact of the current campaign:**_
8. **contact**: contact communication type (categorical: 'cellular','telephone')
9. **month**: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10. **day_of_week**: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11. **duration**: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.

_**Other attributes:**_
12. **campaign**: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13. **pdays**: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14. **previous**: number of contacts performed before this campaign and for this client (numeric)
15. **poutcome**: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')

_**Social and economic context attributes:**_
16. **emp.var.rate**: employment variation rate - quarterly indicator (numeric)
17. **cons.price.idx**: consumer price index - monthly indicator (numeric)
18. **cons.conf.idx**: consumer confidence index - monthly indicator (numeric)
19. **euribor3m**: euribor 3 month rate - daily indicator (numeric)
20. **nr.employed**: number of employees - quarterly indicator (numeric)

_**Output variable (desired target):**_
21. **y** - has the client subscribed a term deposit? (binary: 'yes','no')

### Variable Notes: 

* There are a lot of categorical variables in this data:
    * we will need to deal with those by creating dummy variables for them
    * this will increase the dimensionality of our models significantly, so we will need to explore methods to reduce that where possible
* There are many "unknown" and other placeholder values. We will deal with those on a variable by variable basis
* **duration** is potentially disruptive for our model for the reasons stated in the notes about the variable
    * we can go ahead and drop that variable now

### Importing data: 

Let's import our data for the first time and start get and idea of how it looks

In [31]:
# import data and specifiy the separator as ';', show the first 5 rows
df = pd.read_csv('bank-additional-full.csv', sep=';')
# show all the columns in the dataset 
pd.set_option('display.max_columns', None)
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [32]:
# check the shape of the data 
df.shape

(41188, 21)

In [33]:
# check the data types and number of entries for each variable
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
age               41188 non-null int64
job               41188 non-null object
marital           41188 non-null object
education         41188 non-null object
default           41188 non-null object
housing           41188 non-null object
loan              41188 non-null object
contact           41188 non-null object
month             41188 non-null object
day_of_week       41188 non-null object
duration          41188 non-null int64
campaign          41188 non-null int64
pdays             41188 non-null int64
previous          41188 non-null int64
poutcome          41188 non-null object
emp.var.rate      41188 non-null float64
cons.price.idx    41188 non-null float64
cons.conf.idx     41188 non-null float64
euribor3m         41188 non-null float64
nr.employed       41188 non-null float64
y                 41188 non-null object
dtypes: float64(5), int64(5), object(11)
memory usa

### Initial Data Notes: 
* There are over 41k data entries and 21 variables, including the target variable 'y'
* It appears that there are no missing values, but we already know there are **many placeholder values**

Let's take a look at the descriptive statistics for our continuous variables

In [37]:
# show the central tendencies for each continuous variable
df.describe()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,41188.00000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000
mean,40.02406,258.285010,2.567593,962.475454,0.172963,0.081886,93.575664,-40.502600,3.621291,5167.035911
std,10.42125,259.279249,2.770014,186.910907,0.494901,1.570960,0.578840,4.628198,1.734447,72.251528
min,17.00000,0.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000
25%,32.00000,102.000000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.344000,5099.100000
50%,38.00000,180.000000,2.000000,999.000000,0.000000,1.100000,93.749000,-41.800000,4.857000,5191.000000
75%,47.00000,319.000000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000
max,98.00000,4918.000000,56.000000,999.000000,7.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000


**Notes:** 
* 'duration' has a lot of variance, but we already know we'll be dropping that variable
* the mean and median of 'campaign' are very close, but the max value is much greater
* the placeholder '999' has significant influence over 'pdays'
* most other variables appear fairly stable and normally distributed. We'll confirm in our exploratory data analysis

# Clean and Prepare Data

In this section we will: 
1. Check for and deal with missing values
2. Inspect and learn more about the values in each variable
3. Inspect placeholder values and determine whether we want to:
    * impute values
    * drop entries or variables
    * leave them alone

## Missing Values

Although our previous inspection of the data appeared to have no values missing it is still a good step to confirm 

In [45]:
# check for any na values in the dataset
df.isna().any()

age               False
job               False
marital           False
education         False
default           False
housing           False
loan              False
contact           False
month             False
day_of_week       False
duration          False
campaign          False
pdays             False
previous          False
poutcome          False
emp.var.rate      False
cons.price.idx    False
cons.conf.idx     False
euribor3m         False
nr.employed       False
y                 False
dtype: bool

Looks like we're good to go. Let's move forward with inspecting each variable more closely

## Value Counts

To better appreciate and more easily analyze our different variables we can first separate them into categorical and continuous groups

In [49]:
# create a list of continous and categortical variables based on data type
cont_vars = []
cat_vars = []

for col in df.columns: 
    if df[col].dtype == 'O':
        cat_vars.append(col)
    else: 
        cont_vars.append(col)

print(f'There are {len(cat_vars)} categorical variables: \n', cat_vars)
print('----' * 8)
print(f'There are {len(cont_vars)} continuous variables: \n', cont_vars)

There are 11 categorical variables: 
 ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome', 'y']
--------------------------------
There are 10 continuous variables: 
 ['age', 'duration', 'campaign', 'pdays', 'previous', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed']


In [68]:
# check the number of unique values and the percent each value represents
# for each categoritcal variable
for v in cat_vars:
    print(f'Unique values for {v}: {df[v].nunique()}\n', 
          round(df[v].value_counts(normalize=True)* 100, 2), '\n')

Unique values for job: 12
 admin.           25.30
blue-collar      22.47
technician       16.37
services          9.64
management        7.10
retired           4.18
entrepreneur      3.54
self-employed     3.45
housemaid         2.57
unemployed        2.46
student           2.12
unknown           0.80
Name: job, dtype: float64 

Unique values for marital: 4
 married     60.52
single      28.09
divorced    11.20
unknown      0.19
Name: marital, dtype: float64 

Unique values for education: 8
 university.degree      29.54
high.school            23.10
basic.9y               14.68
professional.course    12.73
basic.4y               10.14
basic.6y                5.56
unknown                 4.20
illiterate              0.04
Name: education, dtype: float64 

Unique values for default: 3
 no         79.12
unknown    20.87
yes         0.01
Name: default, dtype: float64 

Unique values for housing: 3
 yes        52.38
no         45.21
unknown     2.40
Name: housing, dtype: float64 

Unique valu

### Variable Value Notes: 
* **job:** There are 12 unique values for job with good distribution among them
    * Less than 1% of values are "unknown" -
    * With such a small number of unknowns and a relatively large number of unique values this does not seem like it will have a major influence on our model so for now we **can leave it alone** and revisit if it appears so later
* **marital:** There are three unique values for this variable with a distribution that seems representative of society
    * There are less than .2% of values shown as "unknown", but differently than before with only 3 unique variables for this data we can likely safely **impute values** by applying the known distribution to the unknown values
    * This will eliminate one unnecessary variable later on when we code dummies
* **education:** Education has 8 unique values with 4.2% unknown
    * With just under 1 in 20 values unknown we should consider **imputing values** for this variable similar to the 'marital' variable
* **default:** This is a binary yes/no variable, but also includes just over 20% "unknown"
    * This would be a large value to impute or drop.
    * With a very small value for "yes" we could likely safely code all "unknowns" as "no"
    * But, this is also interesting considering the business problem - it would be interesting to know if lack of knowledge of a default actually positively or negatively contributes to a successful marketing call
    * We will **leave "unknown" in place** for now
* **housing:** This is also a binary yes/no variable, but with a much smaller, 2.4%, amount of unknowns
    * For this variable we can likely safely **impute** again by applying the known distribution of yes/no
* **loan:** Another binary yes/no variable with the same % of unknowns
    * We can treat it similarly as the housing variable and **impute values**
    * With the same amount of unknowns it would be interesting to see if they are the same entries, and if they are missing more data should perhaps be dropped altogether
* **contact:** This variable is interestingly binary as well, between "cellular" and "telephone", we'll have to pay attention to how it is coded when we transform to dummies
* **month:** It appears this campaign was only for 10 months and heavily focused on the spring and early summer
* **day_of_week:** Calls were only made during the work week and seem very evenly distributed
    * Given the even distribution it will be very interesting to see if one day of the week was more fruitful that others
* **poutcome:** It appears that most people that had been contacted were not part of a previous campaign
    * For those that were previously contacted it will be very interesting to see if it has an influence on future success - especially those that were successfully reached previously
* **y:** Our target variable. It is good that we have no missing values here! Also it is good that we have a fairly significant positive rate with 11.27%.

**Continuous variables:** We've already previously seen the descriptive statistics of our continuous variables above and with continue to explore them in our EDA. 
* We do however know that we need to deal with the "999" placeholder for "pdays" and drop "duration"

## Dealing with "duration", "unknown", and "999" 

### Duration: 
As stated in the notes for the dataset, the "duration" variable is not a good predictor of a potential "yes" because a 0 is an automatic "no" and time spent before a call cannot be known. We will therefor first drop this from our data set 

In [69]:
# crete a new dataframe for cleaning and drop the 'duration' column
data = df.drop(columns=['duration'], axis=1)
data.head(2)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### "Unknown"
In the above section we decided we would: 
* Impute values for: 
    1. marital
    2. education
    3. housing
    4. loan

# Exploratory Data Analysis and Feature Transformation

age :  78
job :  12
marital :  4
education :  8
default :  3
housing :  3
loan :  3
contact :  2
month :  10
day_of_week :  5
duration :  1544
campaign :  42
pdays :  27
previous :  8
poutcome :  3
emp.var.rate :  10
cons.price.idx :  26
cons.conf.idx :  26
euribor3m :  316
nr.employed :  11
y :  2


## Categorical Variables

## Continuous Variables

## Transformations

# Model Data

## Logistic Regression

## K-Nearest Neighbors

## Random Forest

## Support Vector Maxhine

## XGBoost

## Principle Component Analysis

# Conclusions

# Graveyard

In [38]:
# cont_vars = []
# cat_vars = []

# for col in df.columns: 
#     if df[col].dtype == 'O':
#         cat_vars.append(col)
#     else: 
#         cont_vars.append(col)

# print(f'There are {len(cat_vars)} categorical variables: \n', cat_vars)
# print('----' * 8)
# print(f'There are {len(cont_vars)} continuous variables: \n', cont_vars)

There are 11 categorical variables: 
 ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome', 'y']
--------------------------------
There are 10 continuous variables: 
 ['age', 'duration', 'campaign', 'pdays', 'previous', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed']


In [39]:
# cat_vars.pop()
# cat_vars

['job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'day_of_week',
 'poutcome']

In [40]:
# for v in cat_vars:
#     print(v+':', '\n', df[v].value_counts(normalize=True), '\n')

job: 
 admin.           0.253035
blue-collar      0.224677
technician       0.163713
services         0.096363
management       0.070992
retired          0.041760
entrepreneur     0.035350
self-employed    0.034500
housemaid        0.025736
unemployed       0.024619
student          0.021244
unknown          0.008012
Name: job, dtype: float64 

marital: 
 married     0.605225
single      0.280859
divorced    0.111974
unknown     0.001942
Name: marital, dtype: float64 

education: 
 university.degree      0.295426
high.school            0.231014
basic.9y               0.146766
professional.course    0.127294
basic.4y               0.101389
basic.6y               0.055647
unknown                0.042027
illiterate             0.000437
Name: education, dtype: float64 

default: 
 no         0.791201
unknown    0.208726
yes        0.000073
Name: default, dtype: float64 

housing: 
 yes        0.523842
no         0.452122
unknown    0.024036
Name: housing, dtype: float64 

loan: 
 no       

In [ ]:
# camps = df.campaign.value_counts(normalize=True)
# camps_index = np.array(camps.index)

# cum_camps = []
# i = 0
# for v in camps: 
#     i += v
#     cum_camps.append(i)

# cum_camps = np.array(cum_camps) 


In [ ]:
# fig = plt.figure(figsize=(6,6))

# plt.plot(camps2, cum_camps)
# plt.show()